In [1]:
import nltk
import pandas as pd
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

C:\Users\faraz\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
df1 = pd.read_excel('Tripadvisor_Dataset.xlsx')
print(df1.shape)
df1 = df1.dropna(subset=['Review Text'])
print(df1.shape)

(20016, 11)
(17769, 11)


In [3]:
df1.to_excel('Tripadvisor_Dataset_Filled.xlsx', index=False)

In [4]:
df1 = pd.read_excel('Tripadvisor_Dataset_Filled.xlsx')
df1.describe()

,Hotel Rating,Number of Contributions,Likes on Review,Review Rating
count,17769.000000,17768.000000,17767.000000,17769.000000
mean,3.810794,39.092920,27.936005,3.840227
std,0.712553,97.499388,77.568205,1.344595
min,1.500000,1.000000,1.000000,1.000000
25%,3.500000,1.000000,2.000000,3.000000
50%,4.000000,6.000000,7.000000,4.000000
75%,4.500000,37.000000,27.000000,5.000000
max,5.000000,3388.000000,4818.000000,5.000000


In [5]:
df1.groupby('Review Rating').describe()

Hotel Rating                                               \
                     count      mean       std  min  25%  50%  75%  max   
Review Rating                                                             
1                   1841.0  3.028517  0.828911  1.5  2.5  3.0  4.0  5.0   
2                   1462.0  3.402873  0.733807  1.5  2.5  3.5  4.0  5.0   
3                   2276.0  3.591169  0.697352  1.5  3.0  4.0  4.0  5.0   
4                   4306.0  3.891663  0.579801  1.5  3.5  4.0  4.5  5.0   
5                   7884.0  4.088343  0.540356  1.5  4.0  4.0  4.5  5.0   

              Likes on Review              ...                  \
                        count       mean   ...     75%     max   
Review Rating                              ...                   
1                      1841.0  16.529603   ...    11.0  1068.0   
2                      1462.0  24.811902   ...    24.0  1616.0   
3                      2276.0  31.680580   ...    32.0  1274.0   
4                      4304.0  35.548095   ...    38.0  4818.0   
5                      7884.0  25.942288   ...    24.0  3107.0   

              Number of Contributions                                         \
                                count       mean         std  min  25%   50%   
Review Rating                                                                  
1                              1841.0  16.953829   64.702098  1.0  1.0   2.0   
2                              1462.0  33.654583   90.075140  1.0  1.0   4.0   
3                              2276.0  50.653779  111.022863  1.0  2.0  14.0   
4                              4305.0  59.411614  129.612934  1.0  2.0  17.0   
5                              7884.0  30.838787   75.912550  1.0  1.0   4.0   

                             
                75%     max  
Review Rating                
1               9.0  1343.0  
2              27.0  1233.0  
3              56.0  1776.0  
4              63.0  3388.0  
5              27.0  1343.0  

[5 rows x 24 columns]

In [6]:
authentic_df = df1[df1['Likes on Review'] > 1]

In [7]:
df_review = list(authentic_df['Review Text'].values)

In [8]:
scores_vadar = []
scores_textblob = []

sid = SentimentIntensityAnalyzer()
for review in df_review:
    ss = sid.polarity_scores(str(review))
    tb = TextBlob(str(review))
    snt = tb.sentiment.polarity
    scores_vadar.append((str(review), ss['compound']))
    scores_textblob.append((str(review), snt))
authentic_df_scores = pd.DataFrame(scores_vadar, columns=['Review Text', 'Score_Vadar'])
scores_tb = pd.DataFrame(scores_textblob, columns=['Review Text', 'Score_Textblob'])

# authentic_df_scores.to_csv('Sentiment_scores_authentic.csv', index=False)

In [13]:
left = authentic_df.set_index('Review Text')
right = authentic_df_scores.set_index('Review Text')
result = left.join(right)

another = scores_tb.set_index('Review Text')

result = result.join(another)
result = result.dropna(subset=['Score_Vadar'])
result = result.dropna(subset=['Score_Textblob'])


In [14]:
result['Vador_Sentiment'] = ''
for i, row in result.iterrows():
    if row['Score_Vadar'] > 0:
        result.loc[i,'Vador_Sentiment'] = 'Good'
    elif row['Score_Vadar'] == 0:
        result.loc[i,'Vador_Sentiment'] = 'Neutral'
    elif row['Score_Vadar'] < 0:
        result.loc[i,'Vador_Sentiment'] = 'Bad'

In [15]:
result['Textblob_Sentiment'] = ''
for i, row in result.iterrows():
    if row['Score_Textblob'] > 0:
        result.loc[i,'Textblob_Sentiment'] = 'Good'
    elif row['Score_Textblob'] == 0:
        result.loc[i,'Textblob_Sentiment'] = 'Neutral'
    elif row['Score_Textblob'] < 0:
        result.loc[i,'Textblob_Sentiment'] = 'Bad'

In [16]:
result['Actual_Sentiment'] = ''
for i, row in result.iterrows():
    if row['Review Rating'] in [3,4,5]:
        result.loc[i,'Actual_Sentiment'] = 'Good'
    elif row['Review Rating'] == 2:
        result.loc[i,'Actual_Sentiment'] = 'Noise'
    else:
        result.loc[i,'Actual_Sentiment'] = 'Bad'


In [17]:
result.to_excel('Dataset_with_actual_and_predicted_labels.xlsx')

In [18]:
vadar_correct_predictions = 0
textblob_correct_predictions = 0
total_predictions = 0
for i, row in result.iterrows():
    if row['Actual_Sentiment'] == 'Good' and row['Textblob_Sentiment'] == 'Good':
        textblob_correct_predictions += 1
    elif row['Actual_Sentiment'] == 'Good' and row['Textblob_Sentiment'] == 'Good':
        textblob_correct_predictions += 1
        
    if row['Actual_Sentiment'] == 'Good' and row['Vador_Sentiment'] == 'Good':
        vadar_correct_predictions += 1
    elif row['Actual_Sentiment'] == 'Bad' and row['Vador_Sentiment'] == 'Bad':
        vadar_correct_predictions += 1
        
    if row['Actual_Sentiment'] == 'Good' or row['Actual_Sentiment'] == 'Bad':
        total_predictions += 1

textblob_accuracy = (textblob_correct_predictions/total_predictions)*100
print('Textblob Accuracy: ', textblob_accuracy)
vadar_accuracy = (vadar_correct_predictions/total_predictions)*100
print('Vader Accuracy: ', vadar_accuracy)

Textblob Accuracy:  83.60565159049649
Vader Accuracy:  88.33372799747416


In [19]:
temp_df = result.groupby('Name').describe()
temp_df

temp_df.to_csv('Data Quality Report.csv')

In [20]:
#precision = tp/tp+fp where fp is comments incorrectly classified as 'Good'
#recall = tp/tp+fn where fn is comments incorrectly classified as not 'Good'(Bad)
true_positives = 0
false_positives = 0
false_negatives = 0

for i, row in result.iterrows():
    if row['Actual_Sentiment'] == 'Good' and row['Vador_Sentiment'] == 'Good':
        true_positives += 1
    if row['Actual_Sentiment'] == 'Bad' and row['Vador_Sentiment'] == 'Good':
        false_positives += 1
    if row['Actual_Sentiment'] == 'Good' and row['Vador_Sentiment'] == 'Bad':
        false_negatives += 1

precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)

F_score = 2*((precision*recall)/(precision+recall))

print('Evaluation metrics for Vader:')
print('Precision: ', round(precision, 3))
print('Recall: ', round(recall, 3))
print('F1 score(Harmonic Mean): ', round(F_score, 3))

Evaluation metrics for Vader:
Precision:  0.96
Recall:  0.945
F1 score(Harmonic Mean):  0.953


In [21]:
#precision = tp/tp+fp where fp is comments incorrectly classified as 'Good'
#recall = tp/tp+fn where fn is comments incorrectly classified as not 'Good'(Bad)
true_positives = 0
false_positives = 0
false_negatives = 0

for i, row in result.iterrows():
    if row['Actual_Sentiment'] == 'Good' and row['Textblob_Sentiment'] == 'Good':
        true_positives += 1
    if row['Actual_Sentiment'] == 'Bad' and row['Textblob_Sentiment'] == 'Good':
        false_positives += 1
    if row['Actual_Sentiment'] == 'Good' and row['Textblob_Sentiment'] == 'Bad':
        false_negatives += 1

precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)

F_score = 2*((precision*recall)/(precision+recall))

print('Evaluation metrics for Textblob:')
print('Precision: ', round(precision, 3))
print('Recall: ', round(recall, 3))
print('F1 score(Harmonic Mean): ', round(F_score, 3))

Evaluation metrics for Textblob:
Precision:  0.954
Recall:  0.957
F1 score(Harmonic Mean):  0.955


In [22]:
F_score = 2*((precision*recall)/(precision+recall))
F_score

0.9550516207565033